In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm, multivariate_normal, invwishart, invgamma
from numpy.linalg import inv

In [89]:
def create_w_columns(i,k,ww):
    if i+1<=k:
        a = np.zeros(k)
        a[:(i+1)] = ww[i,:(i+1)]
        return a.T
    else:
        return ww[i,:]
        
create_w_columns(1,2,ww)

array([-2.,  2.])

In [90]:
nsim_data = 10
J = 4
K = 2
zz = multivariate_normal.rvs(mean = np.zeros(K), size=nsim_data)
ww = np.array([[1, 3], [-2, 2],[-1,-1],[2,1]])
sigma = np.array([1,2,.3,.8])
Sigma = np.diag(sigma)
ee = multivariate_normal.rvs(mean = np.zeros(J), cov = Sigma, size=nsim_data)
yy = zz@ww.T + ee

In [91]:
data = dict()
data['N'] = nsim_data
data['K'] = K
data['J'] = J
data['sigma'] = sigma
data['Sigma'] = Sigma
data['y'] = yy
data['z'] = zz
data['w'] = ww
data['e'] = ee

In [66]:
print("dim(w) = ", ww.shape)
print("dim(z) = ", zz.shape)
print("dim(y) = ", yy.shape)
print("dim(e) = ", ee.shape)


dim(w) =  (4, 2)
dim(z) =  (10, 2)
dim(y) =  (10, 4)
dim(e) =  (10, 4)


In [67]:
# sample z
z_s = np.empty_like(zz)
for t in range(data['N']):
    inv_Sigma = inv(Sigma)
    cov1 = inv(np.eye(K) +  ww.T@inv_Sigma@ww)
    mean = cov1 @ ww.T@inv_Sigma@yy[t]
    z_s[t] = multivariate_normal.rvs(mean, cov1)


In [68]:
# sample s
sigma_s = np.empty_like(sigma)
for i in range(data['J']):
    a0 = 1
    b0 = 1
    a = (data['N'] + a0 )/2.
    aux = yy[:,i] - zz@create_w_columns(i,k=2, ww=ww).T
    d = aux.T @ aux
    b = (a0*b0**2+d)/2.
    sigma_s[i] = invgamma.rvs(a, scale = b0)

In [123]:
def trunc_normal(i, mean, cov, max_counter=10):
    if mean.shape[0] == 1:
        a = 0.
        counter = 0
        while a<=0.:
            if counter >= max_counter:
                return 0.
            else:
                a = multivariate_normal.rvs(mean, cov)
                counter+=1
        return a
    
    else:
        a = np.zeros(mean.shape[0])
        counter = 0
        while a[i]<=0.:
            if counter >= max_counter:
                return np.zeros(mean.shape[0])
            else:
                a = multivariate_normal.rvs(mean, cov)
                counter += 1
        return a

trunc_normal(0, np.zeros(1), np.eye(1))

0.19575609890158707

In [116]:
i = 0
if (i+1)<=K:
    print(i, "<K")
    aux1= zz[:,:(i+1)].T@zz[:,:(i+1)]
    inv_C = C0**(-1)*np.eye(i+1) + sigma[i]**(-2)*aux1
    C = inv(inv_C)
    aux2= zz[:,:(i+1)].T@yy[:,i]
    mean = C@(C0**(-1)*np.ones(i+1)+sigma[i]**(-2)*aux2 )
print(mean.shape)

0 <K
(1,)


In [117]:
mean

array([1.20603825])

In [124]:
# sample w
C0 = 1e2
ww_s = np.zeros(ww.shape)
for i in range(J):
    if (i+1)<=K:
        print(i, "<K")
        aux1= zz[:,:(i+1)].T@zz[:,:(i+1)]
        inv_C = C0**(-1)*np.eye(i+1) + sigma[i]**(-2)*aux1
        C = inv(inv_C)
        aux2= zz[:,:(i+1)].T@yy[:,i]
        mean = C@(C0**(-1)*np.ones(i+1)+sigma[i]**(-2)*aux2 )
        print(mean.shape)
        ww_s[i,:(i+1)] = trunc_normal(i, mean, C)
    else:
        print(i)
        aux1= zz.T@zz
        inv_C = C0**(-1)*np.eye(K) + sigma[i]**(-2)*aux1
        C = inv(inv_C)
        aux2= zz.T@yy[:,i]
        mean = C@(C0**(-1)*np.ones(K)+sigma[i]**(-2)*aux2 )
        ww_s[i,:] = multivariate_normal.rvs(mean, cov=C)


0 <K
(1,)
1 <K
(2,)
2
3


In [125]:
def mcmc(data, nsim=100):

    w_s = np.zeros((nsim, data['J'], data['K']))
    sigma_s = np.empty((nsim, data['J']))
    z_s = np.empty((nsim, data['N'], data['K']))

    sigma_temp = np.ones(data['J'])
    Sigma_temp = np.diag(sigma_temp)

    ww_temp = norm.rvs(size=data['J']*data['K']).reshape((data['J'],data['K']))

    C0 = 1e2
    a0 = 1
    b0 = 1

    for j in range(nsim):
        # sample z
        zz_temp = np.empty((data['N'], data['K']))
        for t in range(data['N']):
            inv_Sigma = inv(Sigma)
            cov1 = inv(np.eye(data['K']) +  ww_temp.T@inv_Sigma@ww_temp)
            mean = cov1 @ ww_temp.T@inv_Sigma@data['y'][t]
            zz_temp[t] = multivariate_normal.rvs(mean, cov1)
        z_s[j] = zz_temp

        # sample w
        for i in range(data['J']):
            if (i+1)<=K:
                aux1= zz_temp[:,:(i+1)].T@zz_temp[:,:(i+1)]
                inv_C = C0**(-1)*np.eye(i+1) + sigma_temp[i]**(-2)*aux1
                C = inv(inv_C)
                aux2= zz_temp[:,:(i+1)].T@data['y'][:,i]
                mean = C@(C0**(-1)*np.ones(i+1)+sigma_temp[i]**(-2)*aux2 )
                ww_temp[i,:(i+1)] = trunc_normal(i, mean, C)
            else:
                aux1= zz_temp.T@zz_temp
                inv_C = C0**(-1)*np.eye(data['K']) + sigma_temp[i]**(-2)*aux1
                C = inv(inv_C)
                aux2= zz_temp.T@data['y'][:,i]
                mean = C@(C0**(-1)*np.ones(data['K'])+sigma_temp[i]**(-2)*aux2 )
                ww_temp[i,:] = multivariate_normal.rvs(mean, cov=C)
        w_s[j] = ww_temp

        # sample s
        for i in range(data['J']):
            a = (data['N'] + a0 )/2.
            aux = data['y'][:,i] - zz_temp@create_w_columns(i,k=2, ww=ww_temp).T
            d = aux.T @ aux
            b = (a0*b0**2+d)/2.
            sigma_temp[i] = invgamma.rvs(a, scale = b0)
        sigma_s[j] = sigma_temp
        
    output = dict()
    output['w'] = w_s
    output['z'] = z_s
    output['sigma'] = sigma_s

    return output

In [132]:
ps = mcmc(data, 1000)

In [133]:
from codebase.plot import *

In [134]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 

plots = []
for i in range(data['J']):
    plots.append(plot_trace(ps['sigma'][:,i],
             true_value=data['sigma'][i],
             title = 'Posterior distribution for sigma(%s)'%(i)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Overlay.Posterior_distribution_for_sigma_left_parenthesis_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_sigma_left_parenthesis_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_sigma_left_parenthesis_2_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_sigma_left_parenthesis_3_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)

In [139]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['K']):
    for j in range(0,data['K']):
            plots.append(plot_trace(ps['w'][:,i,j],
                     true_value=data['w'][i,j],
                     title = 'Posterior distribution for w(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Overlay.Posterior_distribution_for_w_left_parenthesis_0_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_w_left_parenthesis_0_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_w_left_parenthesis_1_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_w_left_parenthesis_1_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)

In [140]:
%%opts Curve {+axiswise} [width=600, height=200, tools=['hover']] 
plots = []
for i in range(data['N']):
    for j in range(0,data['K']):
            plots.append(plot_trace(ps['z'][:,i,j],
                     true_value=data['z'][i,j],
                     title = 'Posterior distribution for z(%s,%s)'%(i,j)))
layout = hv.Layout(plots)
layout.cols(1)

:Layout
   .Overlay.Posterior_distribution_for_z_left_parenthesis_0_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_0_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_1_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_1_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_2_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_2_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_3_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_3_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_4_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_4_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_5_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_5_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_6_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_6_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_7_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_7_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_8_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_8_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_9_comma_0_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)
   .Overlay.Posterior_distribution_for_z_left_parenthesis_9_comma_1_right_parenthesis :Overlay
      .Curve.Samples    :Curve   [x]   (y)
      .Curve.True_Value :Curve   [x]   (y)